# Notebook for Ibis experiments

Please note that this notebook require the table *prova*, which can be created inside the *pyspark_experiments.ipynb* notebook.

In [1]:
from poor_man_lakehouse.dremio.builder import DremioConnection
from poor_man_lakehouse.ibis.builder import IbisConnection

d = DremioConnection()

conn = IbisConnection()

/Users/graziano/GitHub/poor-man-lakehouse/.venv/lib/python3.12/site-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(
25/09/20 20:52:24 WARN Utils: Your hostname, MBP-di-Graziano.homenet.telecomitalia.it resolves to a loopback address: 127.0.0.1; using 192.168.1.81 instead (on interface en0)
25/09/20 20:52:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/graziano/apache-spark/3.5.6/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/graziano/.ivy2/cache
The jars for the packages stored in: /Users/graziano/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.13 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.13 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-19b5a528-a4f1-44dd-84c6-a44f8efad70f;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.13;1.9.2 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.13;0.103.3 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.9.2 in central
:: resolution report :: resolve 90ms :: artifacts dl 3ms
	:: modules in use:
	org.apache.iceberg#iceberg-aws-bundle;1.9.2 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.13;1.9.2 from central in [default]
	org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2

In [2]:
print(conn.connections["duckdb"].list_catalogs())
print(conn.connections["duckdb"].list_databases(catalog="memory"))
print(conn.connections["duckdb"].list_tables(database="default"))

['memory', 'system', 'temp']
['main']
[]


In [3]:
conn.set_current_database("default", "duckdb")

In [5]:
polars_table = conn.read_table("default", "prova", "polars")
polars_table.execute()

,datetime,symbol,bid,ask,details
0,2023-01-01 11:00:00+00:00,AAPL,150.0,151.0,"[{'key': 'created_by', 'value': 'user1'}]"
1,2023-01-02 11:00:00+00:00,GOOGL,2800.0,2805.0,"[{'key': 'created_by', 'value': 'user2'}]"
2,2023-01-03 11:00:00+00:00,MSFT,300.0,305.0,"[{'key': 'created_by', 'value': None}]"


In [6]:
conn.sql("select * from default.prova", "duckdb").execute()

,datetime,symbol,bid,ask,details
0,2023-01-01 11:00:00+00:00,AAPL,150.0,151.0,"[{'key': 'created_by', 'value': 'user1'}]"
1,2023-01-02 11:00:00+00:00,GOOGL,2800.0,2805.0,"[{'key': 'created_by', 'value': 'user2'}]"
2,2023-01-03 11:00:00+00:00,MSFT,300.0,305.0,"[{'key': 'created_by', 'value': None}]"


In [7]:
polars_table.select("datetime", "symbol").filter(
    polars_table["symbol"] == "AAPL"
).execute()


,datetime,symbol
0,2023-01-01 11:00:00+00:00,AAPL


In [8]:
pyspark_table = conn.read_table("default", "prova", "pyspark")
pyspark_table.execute()

/Users/graziano/GitHub/poor-man-lakehouse/.venv/lib/python3.12/site-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


,datetime,symbol,bid,ask,details
0,2023-01-01 11:00:00,AAPL,150.0,151.0,{'created_by': 'user1'}
1,2023-01-02 11:00:00,GOOGL,2800.0,2805.0,{'created_by': 'user2'}
2,2023-01-03 11:00:00,MSFT,300.0,305.0,{'created_by': None}


In [9]:
pyspark_table.select("datetime", "symbol").filter(
    pyspark_table["symbol"] == "AAPL"
).execute()


/Users/graziano/GitHub/poor-man-lakehouse/.venv/lib/python3.12/site-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(
25/09/20 20:53:59 WARN SimpleFunctionRegistry: The function unwrap_json_str replaced a previously registered function.
25/09/20 20:53:59 WARN SimpleFunctionRegistry: The function unwrap_json_int replaced a previously registered function.
25/09/20 20:53:59 WARN SimpleFunctionRegistry: The function unwrap_json_bool replaced a previously registered function.
25/09/20 20:53:59 WARN SimpleFunctionRegistry: The function unwrap_json_float replaced a previously registered function.


,datetime,symbol
0,2023-01-01 11:00:00,AAPL


In [10]:
pyspark_table.sql("""SELECT * FROM prova WHERE symbol = 'AAPL'""").execute()

/Users/graziano/GitHub/poor-man-lakehouse/.venv/lib/python3.12/site-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(
25/09/20 20:53:59 WARN SimpleFunctionRegistry: The function unwrap_json_str replaced a previously registered function.
25/09/20 20:53:59 WARN SimpleFunctionRegistry: The function unwrap_json_int replaced a previously registered function.
25/09/20 20:53:59 WARN SimpleFunctionRegistry: The function unwrap_json_bool replaced a previously registered function.
25/09/20 20:53:59 WARN SimpleFunctionRegistry: The function unwrap_json_float replaced a previously registered function.


,datetime,symbol,bid,ask,details
0,2023-01-01 11:00:00,AAPL,150.0,151.0,{'created_by': 'user1'}


In [12]:
conn.list_tables("polars")

['default.prova']

In [13]:
conn._fix_polars_table_name(
    "select * from default.prova join default.prova2 on default.prova.id = default.prova2.id"
)

"select * from 'default.prova' join 'default.prova'2 on 'default.prova'.id = 'default.prova'2.id"

In [14]:
conn.sql("select * from default.prova", "polars").execute()

,datetime,symbol,bid,ask,details
0,2023-01-01 11:00:00+00:00,AAPL,150.0,151.0,"[{'key': 'created_by', 'value': 'user1'}]"
1,2023-01-02 11:00:00+00:00,GOOGL,2800.0,2805.0,"[{'key': 'created_by', 'value': 'user2'}]"
2,2023-01-03 11:00:00+00:00,MSFT,300.0,305.0,"[{'key': 'created_by', 'value': None}]"
